In [2]:
import numpy as np
from pathlib import Path
from typing import Dict
import pandas as pd

In [6]:
# --- Constants (verbatim paths) ---
REPO = r"C:\Users\Joseph\generative-health-models"
FOLD_DIR = REPO + r"\data\processed\tc_multigan_fold_S10"
REAL_SPLIT_DIR = Path(REPO + r"\results\evaluation\real_3class_split_shadow_wesad")
EVAL_DIR = Path(REPO + r"\results\evaluation\3class_cls_only")

FS_ECG = 175.0
FS_LOW = 4.0

# Mapping raw IDs -> 3-class labels (exclude 4=meditation)
USE_IDS = np.array([1, 2, 3], dtype=np.int64)     # keep these raw IDs
REMAPPED = {3: 0, 2: 1, 1: 2}                     # 0=baseline, 1=stress, 2=amusement
ORDER = ["baseline", "stress", "amusement"]


In [7]:
# --- Imports from repo ---
import sys
sys.path.insert(0, str(Path(REPO) / "src"))
from src.evaluation.wesad_real import RealWESADPreparer
from src.evaluation.wesad_eval import WESADEvaluator, EvalConfig

In [8]:
# --- Prepare / rebuild real split with schema {signals, labels, channels} ---
REAL_SPLIT_DIR.mkdir(parents=True, exist_ok=True)
prep = RealWESADPreparer(FOLD_DIR)
X_real_full, y_real_full = prep.prepare(target="ecg")   # (N, 5250, 3), y: (N,)
assert y_real_full is not None, "test_cond.npy missing → labels unavailable."
assert len(X_real_full) == len(y_real_full), "Label count doesn’t match X count."

keep = np.isin(y_real_full, USE_IDS)
remap = np.full_like(y_real_full, fill_value=-1)
for raw, mapped in REMAPPED.items():
    remap[y_real_full == raw] = mapped
X3 = X_real_full[keep]
y3 = remap[keep].astype(np.int64)

def _save_one(label_id: int, name: str) -> Path:
    m = (y3 == label_id)
    return prep.save_npz(
        signals=X3[m].astype(np.float32, copy=False),
        labels=np.full(int(m.sum()), label_id, dtype=np.int32),
        out_dir=REAL_SPLIT_DIR,
        filename=f"real_{name}.npz",
    )

p_baseline = _save_one(0, "baseline")
p_stress   = _save_one(1, "stress")
p_amuse    = _save_one(2, "amusement")

real_files: Dict[str, Path] = {
    "baseline": p_baseline,
    "stress": p_stress,
    "amusement": p_amuse,
}

synth_files = real_files

In [9]:
EVAL_DIR.mkdir(parents=True, exist_ok=True)
cfg = EvalConfig(
    fs_ecg=FS_ECG,
    fs_low=FS_LOW,
    results_dir=EVAL_DIR,
    run_classifier=True,
    clf_labels=(0, 1, 2),
    clf_epochs=10,
    clf_batch_size=64,
    clf_seed=0,
    clf_lr=1e-3,
)

ev = WESADEvaluator(cfg)
results = ev.evaluate_all(real_files=real_files, synth_files=synth_files)

print("Table 1 CSV ->", results.get("table1_csv"))
print("Table 2 CSV ->", results.get("table2_csv"))
print("PSD Figure   ->", results.get("figure_psd"))
print("ACF Figure   ->", results.get("figure_acf"))

# --- Show only Real→Real row ---
t2 = Path(results["table2_csv"])
df = pd.read_csv(t2)
rr = df[df["setting"].astype(str).str.contains("Real→Real", na=False)]
print("\n[Classifier: Real→Real]")
print(rr.to_string(index=False))

Table 1 CSV -> C:\Users\Joseph\generative-health-models\results\evaluation\3class_cls_only\table1_distribution_psd_acf.csv
Table 2 CSV -> C:\Users\Joseph\generative-health-models\results\evaluation\3class_cls_only\table2_classifier_metrics.csv
PSD Figure   -> C:\Users\Joseph\generative-health-models\results\evaluation\3class_cls_only\figure_psd_overlay.png
ACF Figure   -> C:\Users\Joseph\generative-health-models\results\evaluation\3class_cls_only\figure_acf_overlay.png

[Classifier: Real→Real]
  setting    AUROC      F1
Real→Real 0.325216 0.21164


In [ ]:
# === Create a z-score "view" for the classifier (leaves originals untouched) ===
from pathlib import Path
import numpy as np
import pandas as pd
import sys

REPO = r"C:\Users\Joseph\generative-health-models"
sys.path.insert(0, str(Path(REPO) / "src"))
from src.evaluation.wesad_eval import WESADEvaluator, EvalConfig

FS_ECG = 175.0
FS_LOW = 4.0

REAL_RAW_DIR   = Path(REPO + r"\results\evaluation\real_3class_split")           # your existing real files
REAL_ZS_DIR    = Path(REPO + r"\results\evaluation\real_3class_split_cls_zscore") # z-score copies for classifier
EVAL_DIR_CLS   = Path(REPO + r"\results\evaluation\3class_cls_only_ZS_applied")

ORDER = ["baseline","stress","amusement"]

def _load_signals_labels(p: Path):
    d = np.load(p, allow_pickle=True)
    X = d["signals"] if "signals" in d.files else d["X"]
    y = d["labels"]  if "labels"  in d.files else d["y"]
    return X.astype(np.float32), y.astype(np.int32)

def build_zscore_view():
    REAL_ZS_DIR.mkdir(parents=True, exist_ok=True)
    # 1) compute per-channel μ/σ over ALL real windows (N*T per channel)
    all_sig = []
    all_lab = []
    for name in ORDER:
        X, y = _load_signals_labels(REAL_RAW_DIR / f"real_{name}.npz")
        all_sig.append(X); all_lab.append(y)
    S = np.concatenate(all_sig, axis=0)  # (N_total, T, 3)
    mu = S.reshape(-1, S.shape[-1]).mean(axis=0)                  # (3,)
    sd = S.reshape(-1, S.shape[-1]).std(axis=0).astype(np.float32)
    sd[sd == 0] = 1.0

    # 2) write standardized copies with consistent schema
    channels = np.array(["ECG","Resp","EDA"], dtype=object)
    for name in ORDER:
        X, y = _load_signals_labels(REAL_RAW_DIR / f"real_{name}.npz")
        Xz = (X - mu.reshape(1,1,3)) / sd.reshape(1,1,3)
        # normalize labels to 0..2 if they were 1..3
        if y.min() == 1:
            y = (y - 1).astype(np.int32)
        np.savez_compressed(
            REAL_ZS_DIR / f"real_{name}.npz",
            signals=Xz.astype(np.float32, copy=False),
            labels=y.astype(np.int32, copy=False),
            channels=channels,
        )
    print("[cls-view] wrote z-score copies to:", REAL_ZS_DIR)
    return mu, sd

def run_classifier_on(view_dir: Path, eval_dir: Path, epochs: int = 20):
    eval_dir.mkdir(parents=True, exist_ok=True)
    real_files = {k: view_dir / f"real_{k}.npz" for k in ORDER}
    synth_files = real_files  # Real→Real sanity check

    cfg = EvalConfig(
        fs_ecg=FS_ECG, fs_low=FS_LOW,
        results_dir=eval_dir,
        run_classifier=True,
        clf_labels=(0, 1, 2),      # we enforce 0..2 above
        clf_epochs=epochs,         # give the tiny CNN more room to fit
        clf_batch_size=64,
        clf_seed=0,
        clf_lr=1e-3,
    )
    ev = WESADEvaluator(cfg)
    res = ev.evaluate_all(real_files=real_
    
    
    files, synth_files=synth_files)

    # Print Real→Real row
    t2 = Path(res["table2_csv"])
    df = pd.read_csv(t2)
    rr = df[df["setting"].astype(str).str.contains("Real→Real", na=False)]
    print("\n[Classifier on z-score view]")
    print(rr.to_string(index=False))
    return res

if __name__ == "__main__":
    build_zscore_view()
    run_classifier_on(REAL_ZS_DIR, EVAL_DIR_CLS, epochs=20)

[cls-view] wrote z-score copies to: C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split_cls_zscore

[Classifier on z-score view]
  setting    AUROC       F1
Real→Real 0.985399 0.327485
